# Read Overpass-API to Python Pandas Dataframe

The [Overpass API](http://wiki.openstreetmap.org/wiki/Overpass_API) provides access to the data behind the Openstreetmaps Map Data.
The [Overpass-Turbo](http://overpass-turbo.eu/) is the easyiest way to test requests and get the correct code to ask the database. Example: Fountains of the city of Hamburg as per https://github.com/water-fountains/proximap/issues/307

In [58]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 200)

191123_122309


In [3]:
# Get yours at: http://boundingbox.klokantech.com/
bbox = [9.657,53.4075,10.2997,53.7365]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

### Construct the Overpass Query String

Request from [Overpass-Turbo](http://overpass-turbo.eu/)

In [46]:
bbox_string = '(%s,%s,%s,%s)' % (minLat, minLon, maxLat, maxLon)

compactOverpassQLstring = f'[out:json][timeout:60];(node[man_made=drinking_fountain]{bbox_string};node[amenity=drinking_water]{bbox_string};node[amenity=fountain]{bbox_string};'
#node[man_made=water_tap]{bbox_string};node[natural=spring]{bbox_string};node[amenity=watering_place]{bbox_string};
#node[amenity=water_point]{bbox_string};node[man_made=water_well]{bbox_string};
compactOverpassQLstring += ');out ;'
compactOverpassQLstring

'[out:json][timeout:60];(node[man_made=drinking_fountain](53.4075,9.657,53.7365,10.2997);node[amenity=drinking_water](53.4075,9.657,53.7365,10.2997);node[amenity=fountain](53.4075,9.657,53.7365,10.2997););out ;'

In [47]:
osmrequest = {'data': compactOverpassQLstring}
osmurl = 'https://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

### Reformat the JSON to fit in a Pandas Dataframe

The JSON can't be directyl imported to a Pandas Dataframe:

Thanks to [unutbu from stackoverflow.com](http://stackoverflow.com/questions/24848416/expected-string-or-unicode-when-reading-json-with-pandas) for fiddling this out!

In [48]:
osmdata = osm.json()
# print out the json
#osmdata

In [49]:
osmelements = osmdata['elements']
#for dct in osmdata:
#    if dct['type']=='way':
#        for key, val in dct['tags'].iteritems():
#            dct[key] = val
#        del dct['tags']
#    else:
#        pass # nodes

### Now put everything to the Pandas Dataframe

In [50]:
osmdf = pd.DataFrame(osmelements)

### Look at the whole Dataframe

In [51]:
osmdf.head(5)

,type,id,lat,lon,tags
0,node,257690357,53.551760,10.020930,{'amenity': 'fountain'}
1,node,268192970,53.549850,9.935483,"{'amenity': 'fountain', 'image': 'http://commo..."
2,node,276284733,53.545274,9.971903,"{'amenity': 'drinking_water', 'drinking_water:..."
3,node,292561393,53.570260,9.980357,{'amenity': 'fountain'}
4,node,297703374,53.580721,9.704032,{'amenity': 'fountain'}


In [52]:
osmdf.tail(5)

,type,id,lat,lon,tags
122,node,6426760687,53.623274,10.077671,{'amenity': 'drinking_water'}
123,node,6487428230,53.563470,10.073625,{'amenity': 'fountain'}
124,node,6542576093,53.558327,9.823864,{'amenity': 'fountain'}
125,node,6680240179,53.706815,9.683915,{'amenity': 'fountain'}
126,node,6699196385,53.620531,10.038560,{'amenity': 'fountain'}


## Export to CSV

In [62]:
osmCsvFileName = "osmFountainsHamburg"+dt.now().strftime(dtFmt)+".csv"
osmdf.to_csv(osmCsvFileName, index=False)



In [63]:
print("wrote '"+osmCsvFileName+"' with "+str(len(osmdf))+" lines")

wrote 'osmFountainsHamburg191123_123112.csv' with 127 lines
